In [ ]:
import cv2
from ultralytics import YOLO
from PIL import ImageFont, ImageDraw, Image
import numpy as np

# YOLO 모델 불러오기 (사람 탐지 전용)
model = YOLO("yolov8n.pt")

# CCTV 영상 주소 (로컬 파일 또는 RTSP/HTTP 스트리밍 URL 가능)
VIDEO_SOURCE = "../data/iho_beach.mp4"

# 혼잡도 기준 (사람 수)
DENSITY_THRESHOLDS = [5, 15]  # 0~4: 낮음, 5~14: 중간, 15+ : 높음

# 혼잡도 레벨 색상 (BGR)
COLOR_MAP = {
    "낮음": (0, 255, 0),    # 초록
    "중간": (0, 255, 255),  # 노랑
    "높음": (0, 0, 255)     # 빨강
}

def get_density_level(count):
    if count < DENSITY_THRESHOLDS[0]:
        return "낮음"
    elif count < DENSITY_THRESHOLDS[1]:
        return "중간"
    else:
        return "높음"

# 한글 폰트 경로 설정 (윈도우 예: 맑은 고딕)
font_path = "C:/Windows/Fonts/malgun.ttf"  # 환경에 맞게 수정 필요
font = ImageFont.truetype(font_path, 30)  # 폰트 크기 30

# 비디오 스트림 열기
cap = cv2.VideoCapture(VIDEO_SOURCE)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO 객체 탐지 (사람 클래스만 추출)
    results = model(frame, classes=[0], imgsz=1280, conf=0.2, iou=0.3)

    # 탐지된 박스 개수 = 사람 수
    person_count = 0
    for r in results:
        for box in r.boxes:
            person_count += 1
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

    # 혼잡도 계산
    density_level = get_density_level(person_count)
    density_color = COLOR_MAP[density_level]

    # OpenCV 이미지를 PIL 이미지로 변환 (BGR->RGB)
    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(frame_pil)

    # PIL로 한글 텍스트 그리기
    draw.text((20, 40), f"인원 수: {person_count}", font=font, fill=(255, 255, 255))
    draw.text((20, 80), f"혼잡도: {density_level}", font=font, fill=density_color[::-1])  # RGB로 바꿔줌

    # PIL 이미지를 다시 OpenCV 이미지로 변환 (RGB->BGR)
    frame = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)

    # 결과 출력
    cv2.imshow("Beach Crowd Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


0: 736x1280 17 persons, 590.1ms
Speed: 15.1ms preprocess, 590.1ms inference, 11.2ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 18 persons, 570.7ms
Speed: 12.2ms preprocess, 570.7ms inference, 5.7ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 19 persons, 419.9ms
Speed: 15.8ms preprocess, 419.9ms inference, 5.8ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 16 persons, 410.2ms
Speed: 17.6ms preprocess, 410.2ms inference, 6.0ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 18 persons, 402.3ms
Speed: 13.9ms preprocess, 402.3ms inference, 6.2ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 17 persons, 398.2ms
Speed: 13.1ms preprocess, 398.2ms inference, 5.8ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 18 persons, 399.4ms
Speed: 12.0ms preprocess, 399.4ms inference, 6.2ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 16 persons, 402.7ms
Speed: 11.9ms preprocess, 402.7ms i